In [4]:
import pandas as pd
import numpy as np
import string
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')


In [7]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
fake['class']=0
true['class']=1

In [10]:
data=pd.concat([fake,true],axis=0)

In [11]:
data.sample(10)

,title,text,subject,date,class
9955,Patriots Fan Who Tracked Down Brady Jersey Sur...,The New England Patriots owner Robert Kraft su...,politics,"Sep 7, 2017",0
2785,Twitter F*cks Trump Up After He Says Senators...,Trump has signed a stream of executive orders ...,News,"January 29, 2017",0
5747,Trump adopts aggressive posture toward Iran af...,WASHINGTON (Reuters) - The White House put Ira...,politicsNews,"February 1, 2017",1
5720,Exclusive: U.S. expected to impose fresh sanct...,WASHINGTON (Reuters) - The United States is ex...,politicsNews,"February 2, 2017",1
146,One Hero Made Donald Trump Panic And Made Twi...,If you were bored on Thursday night at around ...,News,"November 3, 2017",0
20462,B*TCH OF BENGHAZI WINS Democrat Nomination” I’...,American citizens will be encouraged when the ...,left-news,"Jun 7, 2016",0
23184,Prof Michel Chossudovsky discusses Hillary Cli...,21st Century Wire says Amid great mainstream ...,Middle-east,"November 5, 2016",0
8825,Options market signals little fear of election...,NEW YORK (Reuters) - The November U.S. preside...,politicsNews,"July 8, 2016",1
11061,OBAMA’S FORMER “Press Liar” Warns Trump Not To...,Former White House Press Secretary Josh Earnes...,politics,"Apr 24, 2017",0
3896,REPORT: Trump’s Mind SHATTERED In The Face Of...,"Going into Election Day, Trump s campaign is w...",News,"November 7, 2016",0


In [12]:
data = data.drop(['title','subject','date'],axis=1)

In [13]:
data.reset_index(inplace=True)

In [14]:
data.drop(['index'],axis=1,inplace=True)

In [15]:
data.sample(5)

,text,class
35576,MELBOURNE (Reuters) - Australian police said o...,1
23214,21st Century Wire says As 21WIRE reported prev...,0
29951,(Reuters) - U.S. President-elect Donald Trump’...,1
34119,WASHINGTON (Reuters) - The State Department ex...,1
24961,WASHINGTON (Reuters) - U.S. Health and Human S...,1


In [16]:
stop_words = set(stopwords.words('english'))

In [17]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # remove URLs
    text = re.sub(r'\@w+|\#','', text)  # remove mentions
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # remove special chars
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

In [18]:
# data['text'] = data['title'] + " " + data['text']
data['text'] = data['text'].apply(clean_text)

In [19]:
# 5. Split dataset
X = data['text']
y = data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# 6. Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [21]:
# 7. Train model
model = LogisticRegression()
model.fit(X_train_vec, y_train)

LogisticRegression()

In [22]:
# 8. Evaluation
y_pred = model.predict(X_test_vec)

In [23]:
model.score(X_test_vec, y_test)

0.9869710467706013

In [24]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9869710467706013

Confusion Matrix:
 [[4669   64]
 [  53 4194]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4733
           1       0.98      0.99      0.99      4247

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [25]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(model,"lr_model.jb")

['lr_model.jb']